In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import minimize
from tqdm import tqdm

In [51]:

# Load and prepare the data (same as before)
df = pd.read_csv('/Users/dr/Documents/GitHub/FixedIncome/STRIPS_data.csv')
latest_date = "2025-02-04"
latest_data = df[df['Date'] == latest_date].iloc[0]

# Extract times, prices, and yields
maturities = []
prices = []
yields = []

for i in range(46):
    maturity_label = df.columns[1 + i*3].split('_')[1]
    time_col = f'Time_{maturity_label}'
    price_col = f'Price_{maturity_label}'
    yield_col = f'Yield_{maturity_label}'
    
    time = latest_data[time_col]
    price = latest_data[price_col]
    yield_val = latest_data[yield_col]
    
    maturities.append(time)
    prices.append(price)
    yields.append(yield_val)

# Convert to numpy arrays
maturities = np.array(maturities)
prices = np.array(prices)
yields = np.array(yields)

# Filter out very short maturities
min_maturity = 0.1
mask = maturities >= min_maturity
filtered_maturities = maturities[mask]
filtered_prices = prices[mask]
filtered_yields = yields[mask]


In [52]:

class BGMMonteCarlo:
    def __init__(self, times, initial_forwards, volatilities, corr_matrix=None):
        self.times = np.array(times)
        self.initial_forwards = np.array(initial_forwards)
        self.volatilities = np.array(volatilities)
        self.N = len(times) - 1
        self.deltas = np.diff(times)
        
        if corr_matrix is None:
            self.corr_matrix = np.eye(self.N)
        else:
            self.corr_matrix = corr_matrix
        
        # Perform Cholesky decomposition for correlated random numbers
        try:
            self.cholesky = np.linalg.cholesky(self.corr_matrix)
        except np.linalg.LinAlgError:
            # If matrix isn't positive definite, use nearest correlation matrix
            self.corr_matrix = self._nearest_correlation_matrix(self.corr_matrix)
            self.cholesky = np.linalg.cholesky(self.corr_matrix)
    
    def _nearest_correlation_matrix(self, corr_matrix):
        """Helper function to ensure positive definite correlation matrix"""
        # Simple approach - add small diagonal adjustment
        n = corr_matrix.shape[0]
        adj = np.eye(n) * 1e-6
        return corr_matrix + adj
    
    def simulate_forward_rates(self, num_simulations, num_steps, dt):
        """Simulate forward rate paths using Monte Carlo"""
        paths = np.zeros((num_simulations, num_steps + 1, self.N))
        paths[:, 0, :] = self.initial_forwards
        
        for t in tqdm(range(1, num_steps + 1), desc="Simulating paths"):
            # Generate correlated random numbers
            z = norm.rvs(size=(num_simulations, self.N))
            correlated_z = np.dot(z, self.cholesky.T)
            
            # Euler discretization of BGM dynamics
            drift = self.calculate_drift(paths[:, t-1, :])
            diffusion = paths[:, t-1, :] * self.volatilities * np.sqrt(dt)
            
            paths[:, t, :] = paths[:, t-1, :] * np.exp(
                (drift - 0.5 * self.volatilities**2) * dt 
                + diffusion * correlated_z
            )
        
        return paths
    
    def calculate_drift(self, current_forwards):
        """Calculate the drift term in the BGM dynamics"""
        drift = np.zeros_like(current_forwards)
        for i in range(self.N):
            for j in range(i + 1):
                rho_ij = self.corr_matrix[i, j]
                term = (self.deltas[j] * current_forwards[:, j] * self.volatilities[i] * 
                       self.volatilities[j] * rho_ij) / (
                           1 + self.deltas[j] * current_forwards[:, j])
                drift[:, i] += term
        return drift
    
    def zero_coupon_bond(self, T, paths, dt):
        """Calculate zero coupon bond price for maturity T"""
        step = min(int(T / dt), paths.shape[1] - 1)
        
        # Calculate discount factor for each path
        discount_factors = np.ones(paths.shape[0])
        for i in range(step + 1):
            t = i * dt
            if t > T:
                break
            discount_factors /= (1 + self.deltas[0] * paths[:, i, 0])  # Simplified
        
        return np.mean(discount_factors) * 100
    
    def price_coupon_bond(self, coupon_rate, maturity, frequency, paths, dt):
        """Price a coupon-bearing bond using simulated paths"""
        coupon = 100 * coupon_rate / frequency
        periods = int(maturity * frequency)
        price = 0
        
        for i in range(1, periods + 1):
            t = i / frequency
            if t > maturity:
                break
            price += coupon * self.zero_coupon_bond(t, paths, dt)
        
        # Add principal payment at maturity
        price += 100 * self.zero_coupon_bond(maturity, paths, dt)
        
        return price / 100  # Normalized price


In [53]:

def calibrate_with_monte_carlo(times, yields, prices, num_simulations=500, num_steps=50):
    """Calibrate BGM parameters using Monte Carlo simulation"""
    # Calculate initial forwards from prices
    initial_forwards = np.diff(-np.log(prices[:-1]/100)) / np.diff(times[:-1])
    
    # Ensure we have matching dimensions
    n_forwards = len(times) - 1
    if len(initial_forwards) > n_forwards:
        initial_forwards = initial_forwards[:n_forwards]
    elif len(initial_forwards) < n_forwards:
        initial_forwards = np.pad(initial_forwards, (0, n_forwards - len(initial_forwards)), 
                                 'constant', constant_values=initial_forwards[-1])
    
    volatilities = np.full(n_forwards, 0.2)
    
    # Simple correlation matrix
    corr_matrix = np.exp(-0.1 * np.abs(np.subtract.outer(times[:-1], times[:-1])))
    
    # Monte Carlo parameters
    dt = times[-1] / num_steps
    
    # Objective function to minimize
    def objective(params):
        # Split params into forwards and volatilities
        forwards = params[:n_forwards]
        vols = params[n_forwards:]
        
        bgm = BGMMonteCarlo(times, forwards, vols, corr_matrix)
        paths = bgm.simulate_forward_rates(num_simulations, num_steps, dt)
        
        # Calculate model yields for maturities
        model_prices = []
        for t in times[1:]:
            zcb_price = bgm.zero_coupon_bond(t, paths, dt)
            model_prices.append(zcb_price)
        
        # Convert to yields
        model_yields = -np.log(np.array(model_prices)/100) / times[1:] * 100
        
        # Calculate MSE against market yields
        mse = np.mean((model_yields - yields[1:]*100)**2)
        return mse
    
    # Bounds for parameters
    bounds = [(0.0001, 0.2) for _ in range(n_forwards)]  # forwards
    bounds += [(0.01, 0.5) for _ in range(n_forwards)]   # volatilities
    
    # Initial parameter vector
    x0 = np.concatenate([initial_forwards, volatilities])
    
    # Optimization
    result = minimize(objective, x0, bounds=bounds, method='L-BFGS-B', 
                      options={'maxiter': 10, 'disp': True})
    
    # Split optimized parameters
    opt_forwards = result.x[:n_forwards]
    opt_vols = result.x[n_forwards:]
    
    return opt_forwards, opt_vols, corr_matrix


In [54]:

# Calibrate the model using Monte Carlo
print("Calibrating BGM model with Monte Carlo...")
opt_forwards, opt_vols, corr_matrix = calibrate_with_monte_carlo(
    filtered_maturities, filtered_yields, filtered_prices, 
    num_simulations=200, num_steps=30
)


Calibrating BGM model with Monte Carlo...


Simulating paths: 100%|██████████| 30/30 [00:00<00:00, 133.98it/s]


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           88     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.68446D+00    |proj g|=  3.00000D-01


Simulating paths: 100%|██████████| 30/30 [00:00<00:00, 134.72it/s]



At iterate    1    f=  2.66962D+00    |proj g|=  3.00000D-01


Simulating paths: 100%|██████████| 30/30 [00:00<00:00, 133.11it/s]


At iterate    2    f=  2.68200D+00    |proj g|=  3.00000D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   88      2     27     89     0     0   3.000D-01   2.682D+00
  F =   2.6819996886744537     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


In [55]:

# Initialize BGM model with calibrated parameters
bgm_mc = BGMMonteCarlo(filtered_maturities, opt_forwards, opt_vols, corr_matrix)

# Simulate paths for pricing
print("\nSimulating forward rate paths...")
dt = filtered_maturities[-1] / 30
paths = bgm_mc.simulate_forward_rates(1000, 30, dt)



Simulating forward rate paths...


Simulating paths: 100%|██████████| 30/30 [00:00<00:00, 79.53it/s]


In [56]:
# Load Bond data
csv_file_path = "/Users/dr/Documents/GitHub/FixedIncome/bond data for pricing analysis.csv"  
bond_df = pd.read_csv(csv_file_path)

# Filter and retain only the necessary columns
bond_data = bond_df[['Security', 'Maturity', 'Coupon Rate', 'Price', 'Coupon Frequency']].copy()

# Safely modify the 'Coupon Rate' column
bond_data.loc[:, 'Coupon Rate'] = bond_data['Coupon Rate'] / 100

# Display the updated DataFrame
print(bond_data)

              Security  Maturity  Coupon Rate       Price  Coupon Frequency
0          3-mo T-bill      0.25      0.00000    4.192500                 1
1   10yr Treasury Bond      9.88      0.04625  104.750000                 2
2    5yr Treasury Bond      4.99      0.04000  101.167969                 2
3    2yr Treasury Bond      2.66      0.00625   92.320312                 2
4             2yr TIPS      3.54      0.02375  104.843750                 2
5    Corporate Bond(A)      7.12      0.05125  101.677000                 2
6    Corporate Bond(B)      4.66      0.08875   62.076000                 2
7  Corporate Bond(BBB)     10.00      0.05500  100.433000                 2


In [57]:
def compute_bond_prices_with_coupons(model, maturities, face_value=100, coupon_rate=0.0, frequency=2):
    """
    Compute bond prices using a term structure model, including optional coupon payments.
    
    Parameters:
    - model: Object with a `zero(t0, T)` method (e.g., BGM model instance)
    - maturities: Array of times to maturity (in years)
    - face_value: Bond face value (default: 100)
    - coupon_rate: Annual coupon rate as a decimal (default: 0.0)
    - frequency: Number of coupon payments per year (default: 2, semi-annual)
    
    Returns:
    - np.array of bond prices
    """
    prices = np.zeros(len(maturities))  # Pre-allocate array for efficiency
    
    for i, tau in enumerate(maturities):
        # Principal price at maturity
        principal_price = model.zero(0, tau) * face_value
        
        # Coupon price (if applicable)
        if coupon_rate > 0 and tau >= 1 / frequency:
            coupon_payment = (coupon_rate / frequency) * face_value
            num_coupons = int(tau * frequency)
            coupon_times = np.linspace(1 / frequency, tau, num_coupons)
            # Vectorized discount factor calculation
            discounts = np.array([model.zero(0, t) for t in coupon_times])
            coupon_price = np.sum(discounts * coupon_payment)
        else:
            coupon_price = 0
        
        prices[i] = coupon_price + principal_price
    
    return prices

In [58]:
# STRIPS (zero-coupon) prices
zero_coupon_prices = compute_bond_prices_with_coupons(bgm_mc, maturities, face_value, 0.0, 2)

# Bond prices across STRIPS maturities
bond_model_prices = {}
for _, row in bond_data.iterrows():
    security = row['Security']
    coupon_rate = row['Coupon Rate']
    frequency = row['Coupon Frequency']
    bond_model_prices[security] = compute_bond_prices_with_coupons(bgm_mc, maturities, face_value, coupon_rate, frequency)

NameError: name 'bgm' is not defined

In [ ]:
plt.figure(figsize=(14, 8))
# STRIPS: Model vs Market
plt.plot(maturities, zero_coupon_prices, 'b-', label='STRIPS Model Price', linewidth=1.5)
plt.plot(maturities, prices, 'b--', label='STRIPS Market Price', linewidth=1.5)

plt.xlabel('Maturity (Years)')
plt.ylabel('Price ($)')
plt.title('BGM Model Prices vs Market Prices')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()